In [23]:
# === BLOQUE 0: Bootstrap de Kedro desde un notebook ===
import os, sys, glob
from pathlib import Path

print("CWD Notebook:", os.getcwd())

# (Opcional) Silenciar telemetría de Kedro en este entorno
os.environ.setdefault("KEDRO_DISABLE_TELEMETRY", "1")

# 1) Detectar raíz del proyecto (carpeta con pyproject.toml y src/)
nb_dir = Path.cwd()
candidatos = [nb_dir / "../..", nb_dir / "..", nb_dir]
project_root = None
for c in candidatos:
    c = Path(c).resolve()
    if (c / "pyproject.toml").exists() and (c / "src").exists():
        project_root = c
        break
if project_root is None:
    raise RuntimeError("❌ No pude localizar la raíz del proyecto (pyproject.toml y src/ no encontrados).")

print("Project root:", project_root)

# 2) Asegurar src/ en sys.path
src_path = str(project_root / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)
print("sys.path ok:", src_path in sys.path)

# 3) Exponer la ruta del proyecto como variable de entorno (útil para algunos plugins)
os.environ["KEDRO_PROJECT_PATH"] = str(project_root)

# 4) Bootstrap del proyecto (inicializa settings y PACKAGE_NAME internos)
from kedro.framework.startup import bootstrap_project
bootstrap_project(project_path=str(project_root))

# 5) Crear sesión y obtener contexto + catálogo (helpers)
from kedro.framework.session import KedroSession

def _new_session():
    """Crea una nueva sesión y devuelve (session, context, catalog)."""
    session = KedroSession.create(str(project_root))
    context = session.load_context()
    catalog = context.catalog
    return session, context, catalog

# Sesión inicial
session, context, catalog = _new_session()
print("Kedro listo ✅")
# Algunas versiones no exponen package_name; mostramos información segura:
try:
    from kedro.framework.project import settings
    print("PACKAGE_NAME:", getattr(settings, "PACKAGE_NAME", "(no disponible)"))
except Exception:
    pass
print("Project path:", getattr(context, "project_path", project_root))

# ==== Utilidades prácticas ====

def reload_context():
    """
    Relee catalog/params tras editar conf/ (útil tras cambios en catalog.yml).
    También reinicia la sesión para evitar estados previos.
    """
    global session, context, catalog
    try:
        session.close()
    except Exception:
        pass
    session, context, catalog = _new_session()
    try:
        ds = list(catalog.list())
        print(f"🔄 Contexto recargado. Datasets disponibles: {len(ds)} (usa catalog.list())")
    except Exception:
        print("🔄 Contexto recargado.")

def run_pipeline(name: str):
    """
    Ejecuta una pipeline por nombre dentro de una sesión NUEVA (evita error de 'run' duplicado).
    """
    with KedroSession.create(str(project_root)) as s:
        print(f"▶️ Ejecutando pipeline: {name}")
        s.run(pipeline_name=name)
    print("✅ Pipeline completada:", name)

# Helpers para métricas: intenta CSVDataSet fijo y si no existe, cae a timestamped
import pandas as pd

def _load_metrics_fixed_or_latest(dataset_name: str, base_dir="data/07_model_output", prefix=None):
    """
    Intenta cargar un dataset de métricas declarado en catalog (CSVDataSet con ruta fija).
    Si el archivo no existe o no está declarado, busca el CSV más reciente con prefijo timestamped.
    """
    # 1) Intentar vía catalog
    try:
        df = catalog.load(dataset_name)
        print(f"📥 Cargado desde catalog: '{dataset_name}' (ruta fija)")
        return df
    except Exception as e:
        print(f"⚠️ No se pudo cargar '{dataset_name}' desde catalog (intentando timestamped). Detalle: {e}")

    # 2) Buscar último timestamped
    if prefix is None:
        prefix = dataset_name  # por convención, usamos el mismo nombre
    base_path = project_root / base_dir
    pattern = str(base_path / f"{prefix}_*.csv")
    archivos = sorted(glob.glob(pattern))
    if not archivos:
        raise FileNotFoundError(
            f"No encontré archivos timestamped '{prefix}_*.csv' en {base_dir}. "
            f"Ejecuta la pipeline o ajusta el prefijo."
        )
    print("⏱️ Usando el más reciente:", archivos[-1])
    return pd.read_csv(archivos[-1])

def load_regression_metrics():
    """
    Carga métricas de regresión intentando primero 'regression_metrics' del catalog y,
    si falla, usa el último 'regression_metrics_YYYYMMDD-HHMMSS.csv'.
    """
    return _load_metrics_fixed_or_latest("regression_metrics", prefix="regression_metrics")

def load_classification_metrics():
    """
    Carga métricas de clasificación intentando primero 'classification_metrics' del catalog y,
    si falla, usa el último 'classification_metrics_YYYYMMDD-HHMMSS.csv'.
    """
    return _load_metrics_fixed_or_latest("classification_metrics", prefix="classification_metrics")

print("ℹ️ Utilidades disponibles: reload_context(), run_pipeline(name), load_regression_metrics(), load_classification_metrics()")

# (Opcional) Pruebas rápidas:
# df = catalog.load("merged_data"); print("merged_data shape:", df.shape)
# run_pipeline("regression"); reg = load_regression_metrics(); reg.head()
# run_pipeline("classification"); clf = load_classification_metrics(); clf.head()


CWD Notebook: c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\proyectomachineml\notebooks
Project root: C:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\proyectomachineml
sys.path ok: True
Kedro listo ✅
PACKAGE_NAME: (no disponible)
Project path: C:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\proyectomachineml
ℹ️ Utilidades disponibles: reload_context(), run_pipeline(name), load_regression_metrics(), load_classification_metrics()


In [28]:
reload_context()
run_pipeline("regression")
run_pipeline("classification")


🔄 Contexto recargado. Datasets disponibles: 15 (usa catalog.list())
▶️ Ejecutando pipeline: regression


[11/01/25 23:44:07] INFO     Kedro project proyectomachineml                                         ]8;id=906595;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\framework\session\session.py\session.py]8;;\:]8;id=496506;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\framework\session\session.py#327\327]8;;\

                    INFO     Using synchronous mode for loading and saving data. Use the    ]8;id=966397;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\runner\sequential_runner.py\sequential_runner.py]8;;\:]8;id=588107;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\runner\sequential_runner.py#67\67]8;;\
                             --async flag for potential performance gains.                                         
                             https://docs.kedro.org/en/stable/nodes_and_pipelines/run_a_pip                        
                             eline.html#load-and-save-asynchronously                                               

                    INFO     Loading data from merged_data (CSVDataSet)...                      ]8;id=767667;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=157734;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Running node: prepare_data_reg: prepare_data_reg([merged_data]) ->         ]8;id=493548;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\pipeline\node.py\node.py]8;;\:]8;id=326269;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\pipeline\node.py#364\364]8;;\
                             [X_train_reg;X_test_reg;y_train_reg;y_test_reg]                                       

...

                    INFO     Running node: evaluate_cls:                                                ]8;id=551088;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\pipeline\node.py\node.py]8;;\:]8;id=611826;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\pipeline\node.py#364\364]8;;\
                             evaluate_cls([rf_cls_model;X_test_cls;y_test_cls]) ->                                 
                             [classification_metrics]                                                              

[11/01/25 23:44:11] INFO     Saving data to classification_metrics (JSONDataSet)...             ]8;id=339467;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=148679;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

                    INFO     Completed 3 out of 3 tasks                                     ]8;id=685272;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\runner\sequential_runner.py\sequential_runner.py]8;;\:]8;id=445753;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\runner\sequential_runner.py#93\93]8;;\

                    INFO     Pipeline execution completed successfully.                               ]8;id=183696;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\runner\runner.py\runner.py]8;;\:]8;id=675050;file://c:\Users\Gamer\Desktop\Machine_learning\Proyectomachinelearning\venv\Lib\site-packages\kedro\runner\runner.py#123\123]8;;\

✅ Pipeline completada: classification
